In [1]:
import os

In [2]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

Entity Definition

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

Configuration Manager

In [7]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

Data Validation Strategy

In [10]:
import os
from Credit_Card_Fraud_Detection import logger

In [53]:
class DataValiadtionStrategy:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            logger.info(all_cols)

            all_schema = self.config.all_schema.keys()

            logger.info(all_schema)

            
            all_schema_set = set(all_schema)
            validation_status = True # Set by default as True
            for col in all_cols:
                if col not in all_schema_set:
                    validation_status = False
                    break  # No need to continue checking if validation fails

            # Write the final validation status
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [54]:
import pandas as pd

In [55]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtionStrategy(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-01-02 05:51:04,291: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-02 05:51:04,293: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-02 05:51:04,297: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-02 05:51:04,298: INFO: common: created directory at: artifacts]
[2024-01-02 05:51:04,299: INFO: common: created directory at: artifacts/data_validation]
[2024-01-02 05:51:05,090: INFO: 1708381967: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']]
[2024-01-02 05:51:05,090: INFO: 1708381967: dict_keys(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class'])]
